# 8 Búsqueda en Texto

La búsqueda de patrones en un texto es un problema muy importante en la práctica. Sus aplicaciones en computación son variadas, como por ejemplo la búsqueda de una palabra en un archivo de texto o problemas relacionados con biología computacional, en donde se requiere buscar patrones dentro de una secuencia de ADN, la cual puede ser modelada como una secuencia de caracteres (el problema es más complejo que lo descrito, puesto que se requiere buscar patrones en donde ocurren alteraciones con cierta probabilidad, esto es, la búsqueda no es exacta).

En este capítulo se considerará el problema de buscar la ocurrencia de un patrón dentro de un texto. Se utilizarán las siguientes convenciones:

* $n$ denotará el largo del texto en donde se buscará el patrón, es decir, el texto es $a_0 a_1 \ldots a_{n-1}$
* $m$ denotará el largo del patrón a buscar, es decir el patrón es $b_0 b_1 \ldots b_{m-1}$

Por ejemplo:

* Texto = "analisis de algoritmos"
* Patrón = "algo"

## Algoritmo de fuerza bruta

Se alinea la primera posición del patrón con la primera posición del texto, y se comparan los caracteres uno a uno hasta que se acabe el patrón, en cuyo caso se encontró una ocurrencia del patrón en el texto, o hasta que se encuentre una discrepancia.

![bruta1](bruta1.gif)

Si se detiene la búsqueda por una discrepancia, se desliza el patrón en una posición hacia la derecha y se intenta calzar el patrón nuevamente.

![bruta2](bruta2.gif)

En el peor caso este algoritmo realiza $\Theta(mn)$ comparaciones de caracteres.

Si suponemos que todos los caracteres aparecen en forma equiprobable e independiente, en el caso esperado el método de fuerza bruta funciona mucho mejor. Si el alfabeto es de tamaño $c$, entonces en cada comparación la probabilidad de que dos caracteres sean iguales es $\frac{1}{c}$ y de que sean distintos es $1-\frac{1}{c}$. A partir de cada posición, el número esperado de comparaciones que se efectúa hasta encontrar un descalce es

$$
\frac{1}{1-\frac{1}{c}}=\frac{c}{c-1}= 1+\frac{1}{c-1}
$$

Por lo tanto, en cada posición en que el patrón no está, el número esperado de comparaciones es una constante poco mayor que $1$, por lo tanto el costo total de la búsqueda es $\Theta(n)$.

## Algoritmo Knuth-Morris-Pratt (KMP)

Suponga que se está comparando el patrón y el texto en una posición dada, cuando se encuentra una discrepancia.

![kmp1](kmp1.png)

Sea $x$ la parte del patrón que calza con el texto, e $y$ la correspondiente parte del texto, y suponga que el largo de $x$ es $j$. El algoritmo de fuerza bruta mueve el patrón una posición hacia la derecha, sin embargo, esto puede o no puede ser lo correcto en el sentido que los primeros $j-1$ caracteres de $x$ pueden o no pueden calzar los últimos $j-1$ caracteres de $y$.

La observación clave que realiza el algoritmo Knuth-Morris-Pratt (en adelante KMP) es que $x$ es igual a $y$, por lo que la pregunta planteada en el párrafo anterior puede ser respondida mirando solamente el patrón de búsqueda, lo cual permite precalcular la respuesta y almacenarla en una tabla.

Por lo tanto, si deslizar el patrón en una posición no funciona, se puede intentar deslizarlo en $2, 3, \ldots,$ hasta $j$ posiciones.

Se define la *función de fracaso* (*failure function*) del patrón como:

$$
f(j)=\max\{i | 0 \le i<j \wedge b_0\ldots b_{i-1} = b_{j-i-1}\ldots b_{j-1} \}
$$

![kmp2](kmp2.png)

Intuitivamente, $f(j)$ es el largo del mayor prefijo de $x$ que además es sufijo del mismo $x$.

Si se detecta una discrepancia entre el patrón y el texto cuando se trata de calzar $b_j$, se desliza el patrón de manera que $b_{f(j)-1}$ se encuentre donde $b_{j-1}$ se encontraba, y se intenta calzar nuevamente.

![kmp3](kmp3.png)

Suponiendo que se tiene la función $f(j)$ precalculado, la implementación del algoritmo KMP es la siguiente:

In [13]:
def kmp(a,b,f): # busca b dentro de a, retorna None (fracaso) o posición del calce
    n=len(a)
    m=len(b)
    j=0
    for k in range(0,n):
        if j==m:
            return k-m # éxito
        while j>0 and a[k]!=b[j]:
            j=f[j]
        if a[k]==b[j]:
            j=j+1
    return None # fracaso

In [14]:
f=[0,0,1,0,1,2,2]
print(kmp("aaaabaabaaabb","aabaaa",f))
print(kmp("aaaabaabaaabb","abbaaa",f))

5
None


El tiempo de ejecución de este algoritmo no es difícil de analizar, pero es necesario ser cuidadoso al hacerlo. Dado que se tienen dos ciclos anidados, se puede acotar el tiempo de ejecución como el número de veces que se ejecuta el ciclo externo (menor o igual a $n$) multiplicado por el número de veces que se ejecuta el ciclo interno (menor o igual a $m$), lo que de una cota superior a $O(mn)$, ¡que es igual a lo que demora el algoritmo de fuerza bruta!.

Sin embargo, el análisis descrito es pesimista. Observemos que el número total de veces que el ciclo interior es ejecutado es menor o igual al número de veces que se puede decrementar $j$, dado que $f(j)<j$. Pero $j$ comienza desde cero y es siempre mayor o igual que cero, por lo que dicho número es menor o igual al número de veces que $j$ es incrementado, el cual es menor que $n$. Por lo tanto, el tiempo total de ejecución es $\Theta(n)$ en el peor caso. Por otra parte, $k$ nunca es decrementado, lo que implica que el algoritmo nunca se devuelve en el texto.

### Cálculo de la función de fracaso

Queda por resolver el problema de definir la función de fracaso, $f(j)$. Esto se puede realizar inductivamente. Para empezar, $f(1)=0$ por definición. Para calcular $f(j+1)$ suponga que ya se tienen almacenados los valores de $f(1), f(2), \ldots, f(j)$. Se desea encontrar un $i$ tal que el $i$-ésimo carácter del patrón sea igual al $j$-ésimo carácter del patrón.

![kmp4](kmp4.gif)

Para esto se debe cumplir que $i=f(j)$. Si $b_i=b_j$, entonces $f(j+1)=i+1$. En caso contrario, se reemplaza $i$ por $f(i)$ y se verifica nuevamente la condición.

El algoritmo resultante es el siguiente (note que es similar al algoritmo KMP):

In [11]:
def fracaso(b): # calcula y retorna función de fracaso para patrón b
    m=len(b)
    f=[0]*(m+1)
    for j in range(1,m):
        i=f[j]
        while i>0 and b[i]!=b[j]:
            i=f[i]
        if b[i]==b[j]:
            f[j+1]=i+1
        else:
            f[j+1]=0
    return f

In [12]:
print(fracaso("aabaaa"))

[0, 0, 1, 0, 1, 2, 2]


El tiempo de ejecución para calcular la función de fracaso puede ser acotado por los incrementos y decrementos de la variable $i$, y por lo tanto es $\Theta(m)$.

Por lo tanto, el tiempo total de ejecución del algoritmo en el peor caso, sumando el preprocesamiento del patrón más la búsqueda, es $\Theta(m+n)$.